In [119]:
from sympy import *
import sympy
from sympy.abc import *

In [120]:
t=Symbol('t', positive=True, integer=True)
w=Function('w')
a,b,c=symbols('a b c')
S=Function('S')
k=Symbol('k')
P=Function('P')
W=Function('W')

eq1=Eq(diff(w(t), t), a*w(t)*(1-w(t)/b)-c*S(t)*w(t))
eq1

Eq(Derivative(w(t), t), a*(1 - w(t)/b)*w(t) - c*S(t)*w(t))

In [121]:
# ICmax=Function('IC_{max}')
LAI=Function('LAI')
ICmax = (0.935)+ LAI(t)*(0.498) - (LAI(t)**2)*(0.00575)
ICmax

-0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935

In [122]:
cp=((1-exp(-eg*w(t)/w_star)) * 
   (alpha_star - W(t)* (1-exp(eg*w(t)/w_star))-1.8*S(t)))
cp

(1 - exp(-\epsilon_g*w(t)/w^*))*(\alpha^* - (1 - exp(\epsilon_g*w(t)/w^*))*W(t) - 1.8*S(t))

In [123]:
ICstore=cp*ICmax*(1-exp(-k*P(t)/ICmax))
ICstore=simplify(ICstore)
ICstore

(exp(\epsilon_g*w(t)/w^*) - 1)*(exp(k*P(t)/(-0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935)) - 1)*(\alpha^* + (exp(\epsilon_g*w(t)/w^*) - 1)*W(t) - 1.8*S(t))*(-0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935)*exp(-\epsilon_g*w(t)/w^* - k*P(t)/(-0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935))

In [124]:
alpha_star=Symbol(r'\alpha^*')
eg=Symbol(r'\epsilon_g')
w_star=Symbol(r'w^*')

In [126]:
E=Function('E')
eq2=Eq(W(t).diff(),
  P(t)-E(t)-ICstore )
eq2

Eq(Derivative(W(t), t), -(exp(\epsilon_g*w(t)/w^*) - 1)*(exp(k*P(t)/(-0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935)) - 1)*(\alpha^* + (exp(\epsilon_g*w(t)/w^*) - 1)*W(t) - 1.8*S(t))*(-0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935)*exp(-\epsilon_g*w(t)/w^* - k*P(t)/(-0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935)) - E(t) + P(t))

In [127]:
eqlist=[eq1,eq2]
eqlist

[Eq(Derivative(w(t), t), a*(1 - w(t)/b)*w(t) - c*S(t)*w(t)),
 Eq(Derivative(W(t), t), -(exp(\epsilon_g*w(t)/w^*) - 1)*(exp(k*P(t)/(-0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935)) - 1)*(\alpha^* + (exp(\epsilon_g*w(t)/w^*) - 1)*W(t) - 1.8*S(t))*(-0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935)*exp(-\epsilon_g*w(t)/w^* - k*P(t)/(-0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935)) - E(t) + P(t))]

In [129]:
dsolve(eq1, w(t))

Eq(w(t), b*exp(a*t - c*Integral(S(t), t))/(C1*b + a*Integral(exp(a*t)*exp(-c*Integral(S(t), t)), t)))

In [130]:
dsolve(eq2, W(t))

KeyboardInterrupt: 